## Build A Simple Chatbot In Python With Deep Learning

##### Note: Our bot is not going to be super-duper intelligent so it would not always recognize what we are saying, but with enough examples, it would do a pretty decent job at deciphering.

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random 
import pandas as pd


In [2]:
#2. Setting variables
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('commands.json').read()
intents = json.loads(data_file)

In [3]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do

In [4]:
# Data pre-processing
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #print (pattern)
        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag'])) # e.g. [(['Hi', 'there'], 'greeting')]
        #print (documents)
        
        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
classes

['greeting',
 'goodbye',
 'thanks',
 'options',
 'adverse_drug',
 'blood_pressure',
 'blood_pressure_search',
 'pharmacy_search',
 'hospital_search']

In [6]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents" , documents)
print ('\n')
print (len(classes), "classes", classes)
print ('\n')
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents [(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'), (['Open', 'adverse', 'drugs', 'module'], 'adverse_drug'),

In [7]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pd.Series(pattern_words).str.lower()]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\HP\AppData\Local\Temp\ipykernel_520\3331034716.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [8]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

C:\Users\HP\anaconda3\envs\NLP\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
10/10 [==============================] - 2s 9ms/step - loss: 2.2211 - accuracy: 0.1702
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1392 - accuracy: 0.2128
Epoch 3/200
10/10 [==============================] - 0s 3ms/step - loss: 2.0684 - accuracy: 0.2128
Epoch 4/200
10/10 [==============================] - 0s 3ms/step - loss: 1.9149 - accuracy: 0.3191
Epoch 5/200
10/10 [==============================] - 0s 3ms/step - loss: 1.9229 - accuracy: 0.3404
Epoch 6/200
10/10 [==============================] - 0s 5ms/step - loss: 1.7615 - accuracy: 0.4043
Epoch 7/200
10/10 [==============================] - 0s 3ms/step - loss: 1.6431 - accuracy: 0.4894
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 1.4589 - accuracy: 0.5745
Epoch 9/200
10/10 [==============================] - 0s 3ms/step - loss: 1.4799 - accuracy: 0.4681
Epoch 10/200
10/10 [==============================] - 0s 3ms/step - loss: 1.2538 - accuracy: 0.5957
Epoch 11/

10/10 [==============================] - 0s 2ms/step - loss: 0.0515 - accuracy: 0.9787
Epoch 84/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0617 - accuracy: 1.0000
Epoch 85/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0663 - accuracy: 0.9787
Epoch 86/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0548 - accuracy: 1.0000
Epoch 87/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0148 - accuracy: 1.0000
Epoch 88/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0837 - accuracy: 0.9574
Epoch 89/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1305 - accuracy: 0.9787
Epoch 90/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0349 - accuracy: 1.0000
Epoch 91/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0853 - accuracy: 0.9787
Epoch 92/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0845 - accuracy: 0.9787
Epoch 93/200


10/10 [==============================] - 0s 2ms/step - loss: 0.0152 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0142 - accuracy: 1.0000
Epoch 166/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0520 - accuracy: 0.9787
Epoch 167/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0111 - accuracy: 1.0000
Epoch 168/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0625 - accuracy: 0.9787
Epoch 169/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0119 - accuracy: 1.0000
Epoch 170/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0142 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 172/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0053 - accuracy: 1.0000
Epoch 173/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0107 - accuracy: 1.0000
Epoc

In [9]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('commands.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [10]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    #print (p)
    res = model.predict(np.array([p]))[0]
    #print (res)
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    #print (ints)
    res = getResponse(ints, intents)
    return res

In [11]:
while True:
    query = input("User (enter quit() to exit the bot) > ")
    query1 = [chatbot_response(query)]
    question = " ".join(query1)
    response = question #k.respond(question)
    if (query=='quit()'):
        print ('bot > Good Bye, see you again !!!')
        break
    
    if response:
        print("bot > ", response)
    else:
        print("bot > :) ", )

User (enter quit() to exit the bot) > hi
1/1 [==============================] - 0s 118ms/step
bot >  Hello, thanks for asking
User (enter quit() to exit the bot) > ho are u
1/1 [==============================] - 0s 19ms/step
bot >  Hi there, how can I help?
User (enter quit() to exit the bot) > need to find hospital near me
1/1 [==============================] - 0s 6ms/step
bot >  Please provide hospital name or location
User (enter quit() to exit the bot) > dehli
1/1 [==============================] - 0s 19ms/step
bot >  Good to see you again
User (enter quit() to exit the bot) > quit()
1/1 [==============================] - 0s 24ms/step
bot > Good Bye, see you again !!!
